**Importa las bibliotecas necesarias.**

In [1]:
import psycopg2
from psycopg2 import Error # Importar la clase Error
from psycopg2 import sql
import pandas as pd

**Función para ejecutar una consulta SQL**

In [2]:
# Función para ejecutar una consulta SQL parametrizada
def execute_query(conn, query, params=None):
    """
    Ejecuta una consulta SQL con manejo de errores y soporte para parámetros.
    """
    try:
        with conn.cursor() as cursor:
            if params:
                cursor.execute(query, params)  # Pasar parámetros correctamente
            else:
                cursor.execute(query)
            
            conn.commit()
            print(f"Query executed successfully: {query[:50]}...")
            
            if cursor.description:
                return cursor.fetchall(), [desc[0] for desc in cursor.description]  # Obtener nombres de columnas
            else:
                return [], []  # Retornar listas vacías si no hay resultados
    except psycopg2.Error as err:
        conn.rollback()
        print(f"Error executing query: {err}")
        return [], []


**Conexión a la base de datos de PostgreSQL data_powergen**

In [3]:
# Conexión a la base de datos
def create_database_connection():
    conn = None
    try:
        conn = psycopg2.connect(
            database="data_powergen", 
            user="datafundamental_owner", 
            password="uozci0wQGpA6", 
            host="ep-solitary-pond-a2p3evpo.eu-central-1.aws.neon.tech", 
            sslmode="require", 
            port=5432
        )
        print("Connected to 'data_powergen' successfully.")
    except psycopg2.Error as err:
        print(f"Error: {err}")
    return conn

**Crea la tabla resultados_join y la lista de consultas a ejecutar.**

In [7]:
query_parametrizada = """
    SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
    FROM historicos_ordenes
    WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
"""

sql_queries = {
    "create_resultados_join": """
        CREATE TABLE IF NOT EXISTS resultados_join AS
        SELECT 
            h.id_equipo,
            COUNT(h.id_orden) AS total_ordenes_mantenimiento,
            AVG(r.temperatura_c) AS promedio_temperatura,
            AVG(r.vibracion_mm_s) AS promedio_vibracion,
            AVG(r.presion_bar) AS promedio_presion
        FROM historicos_ordenes h
        LEFT JOIN registros_condiciones r
        ON h.id_equipo = r.id_equipo
        GROUP BY h.id_equipo;
    """,
    "total_ordenes_por_equipo": """
        SELECT id_equipo, COUNT(id_orden) AS total_ordenes
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "duracion_promedio_vida_util": """
        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util
        FROM caracteristicas_equipos;
    """,
    "costo_promedio_correctivo": """
        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo';
    """,
    "equipo_mas_fallas": """
        SELECT id_equipo, COUNT(id_orden) AS total_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo'
        GROUP BY id_equipo
        ORDER BY total_correctivo DESC
        LIMIT 1;
    """,
    "temperatura_resumen": """
        SELECT id_equipo, AVG(temperatura_c) AS promedio_temp, MAX(temperatura_c) AS max_temp
        FROM registros_condiciones
        GROUP BY id_equipo;
    """,
    "ordenes_correctivas_equipo": """
        SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
    """,
    "dias_vibracion_temperatura": """
        SELECT *
        FROM registros_condiciones
        WHERE vibracion_mm_s > 3.0 AND temperatura_c > 60;
    """,
    "tiempo_promedio_mantenimiento": """
        WITH diferencias AS (
            SELECT 
                id_equipo,
                EXTRACT(EPOCH FROM AGE(fecha, LAG(fecha) OVER (PARTITION BY id_equipo ORDER BY fecha))) / 3600 AS tiempo_entre_mantenimientos
            FROM historicos_ordenes
        )
        SELECT 
            id_equipo, 
            AVG(tiempo_entre_mantenimientos) AS tiempo_promedio_horas
        FROM diferencias
        GROUP BY id_equipo;
    """,
    "suma_horas_acumuladas": """
        SELECT id_equipo, SUM(horas_acumuladas) AS total_horas_acumuladas
        FROM registros_condiciones
        GROUP BY id_equipo;
    """
}



**Ejecutar las consultas y mostrar resultados ordenados**

In [ ]:
# Ejecutar las consultas y mostrar resultados ordenados
if __name__ == "__main__":
    conn = create_database_connection()
    if conn:
        equipo_id = ('E005',)  # Parámetro debe ser tupla para psycopg2
        print("Executing query: ordenes_correctivas_equipo")
        result, columns = execute_query(conn, query_parametrizada, equipo_id)

        
        for query_name, query in sql_queries.items():
            print(f"Executing query: {query_name}")
            result, columns = execute_query(conn, query)
            if result and columns:
                df = pd.DataFrame(result, columns=columns)
                df_sorted = df.sort_values(by=df.columns[0])  # Ordenar por la primera columna
                print(f"Resultados de '{query_name}':")
                display(df_sorted)  # Mostrar tabla ordenada en el notebook
            else:
                print(f"No data returned for query: {query_name}")

        # Cerrar conexión
        conn.close()
        print("Conexión cerrada.")

In [9]:
import psycopg2
from psycopg2 import sql, Error
import pandas as pd

# Función para ejecutar una consulta SQL con parámetros
def execute_query(conn, query, params=None):
    """
    Ejecuta una consulta SQL con manejo de errores y soporte para parámetros.
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql.SQL(query), params)  # Pasar la consulta como SQL seguro
            conn.commit()
            print(f"Query executed successfully: {query[:50]}...")

            if cursor.description:
                return cursor.fetchall(), [desc[0] for desc in cursor.description]  # Obtener nombres de columnas
            else:
                return [], []  # Retornar listas vacías si no hay resultados
    except psycopg2.Error as err:
        conn.rollback()
        print(f"Error executing query: {err}")
        return [], []

# Función para conectar a la base de datos
def create_database_connection():
    try:
        conn = psycopg2.connect(
            database="data_powergen", 
            user="datafundamental_owner", 
            password="uozci0wQGpA6", 
            host="ep-solitary-pond-a2p3evpo.eu-central-1.aws.neon.tech", 
            sslmode="require", 
            port=5432
        )
        print("Connected to 'data_powergen' successfully.")
        return conn
    except psycopg2.Error as err:
        print(f"Error: {err}")
        return None

# Consulta parametrizada utilizando el módulo sql de psycopg2
query_parametrizada = """
    SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
    FROM historicos_ordenes
    WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
"""
sql_queries = {
    "create_resultados_join": """
        CREATE TABLE IF NOT EXISTS resultados_join AS
        SELECT 
            h.id_equipo,
            COUNT(h.id_orden) AS total_ordenes_mantenimiento,
            AVG(r.temperatura_c) AS promedio_temperatura,
            AVG(r.vibracion_mm_s) AS promedio_vibracion,
            AVG(r.presion_bar) AS promedio_presion
        FROM historicos_ordenes h
        LEFT JOIN registros_condiciones r
        ON h.id_equipo = r.id_equipo
        GROUP BY h.id_equipo;
    """,
    "total_ordenes_por_equipo": """
        SELECT id_equipo, COUNT(id_orden) AS total_ordenes
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "duracion_promedio_vida_util": """
        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util
        FROM caracteristicas_equipos;
    """,
    "costo_promedio_correctivo": """
        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo';
    """,
    "equipo_mas_fallas": """
        SELECT id_equipo, COUNT(id_orden) AS total_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo'
        GROUP BY id_equipo
        ORDER BY total_correctivo DESC
        LIMIT 1;
    """,
    "temperatura_resumen": """
        SELECT id_equipo, AVG(temperatura_c) AS promedio_temp, MAX(temperatura_c) AS max_temp
        FROM registros_condiciones
        GROUP BY id_equipo;
    """,
    "ordenes_correctivas_equipo": """
        SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
    """,
    "dias_vibracion_temperatura": """
        SELECT *
        FROM registros_condiciones
        WHERE vibracion_mm_s > 3.0 AND temperatura_c > 60;
    """,
    "tiempo_promedio_mantenimiento": """
        WITH diferencias AS (
            SELECT 
                id_equipo,
                EXTRACT(EPOCH FROM AGE(fecha, LAG(fecha) OVER (PARTITION BY id_equipo ORDER BY fecha))) / 3600 AS tiempo_entre_mantenimientos
            FROM historicos_ordenes
        )
        SELECT 
            id_equipo, 
            AVG(tiempo_entre_mantenimientos) AS tiempo_promedio_horas
        FROM diferencias
        GROUP BY id_equipo;
    """,
    "suma_horas_acumuladas": """
        SELECT id_equipo, SUM(horas_acumuladas) AS total_horas_acumuladas
        FROM registros_condiciones
        GROUP BY id_equipo;
    """
}
# Ejecutar las consultas con parámetros
if __name__ == "__main__":
    conn = create_database_connection()
    if conn:
        equipo_id = ('E005',)  # Parámetro debe ser tupla para psycopg2

        print("Executing query: ordenes_correctivas_equipo")
        result, columns = execute_query(conn, query_parametrizada, equipo_id)

        if result and columns:
            # Convertir a DataFrame y ordenar por costo de mantenimiento
            df = pd.DataFrame(result, columns=columns)
            df_sorted = df.sort_values(by='costo_mantenimiento', ascending=False)

            print("Resultados de la consulta parametrizada ordenados:")
            display(df_sorted)  # Mostrar resultados ordenados en Azure Notebook
        else:
            print("No data returned for query.")

        conn.close()
        print("Conexión cerrada.")


Connected to 'data_powergen' successfully.
Executing query: ordenes_correctivas_equipo
Query executed successfully: 
    SELECT id_orden, id_equipo, costo_mantenimien...
Resultados de la consulta parametrizada ordenados:


,id_orden,id_equipo,costo_mantenimiento,tiempo_reparacion_horas
184,OT007244,E005,4984,3
80,OT003288,E005,4969,34
19,OT000671,E005,4956,33
64,OT002766,E005,4948,29
237,OT009182,E005,4946,44
...,...,...,...,...
251,OT009941,E005,577,2
70,OT002883,E005,577,16
29,OT001226,E005,556,15
222,OT008368,E005,503,41


Conexión cerrada.
